# Merging Climate Datasets Answer Key

This notebook demonstrates how to merge two gridded climate datasets that differ in their spatial and temporal resolutions using `xarray`. We will:

- Access two public NOAA datasets from THREDDS servers
- Harmonize their coordinates and resolutions with `xr.resample` and `xr.interp`
- Merge them into a single dataset with `xr.merge` for joint analysis


## 1. Imports and configuration


In [ ]:
import matplotlib.pyplot as plt
import xarray as xr

try:
    import cartopy.crs as ccrs
    import cartopy.feature as cfeature
except ImportError:
    ccrs = None
    cfeature = None

TEMP_URL = "https://psl.noaa.gov/thredds/dodsC/Datasets/ncep.reanalysis/surface/air.sig995.2020.nc"
PRECIP_URL = "https://psl.noaa.gov/thredds/dodsC/Datasets/cpc_global_precip/precip.2020.nc"

LAT_RANGE = (20, 50)  # degrees North
LON_RANGE_360 = (230, 300)  # degrees East (equivalent to -130° to -60°)
LON_RANGE_180 = (-130, -60)  # degrees relative to Greenwich

TIME_RANGE = slice("2020-06-01", "2020-06-30")


The two datasets we will use are:

- **NCEP/NCAR Reanalysis 1** surface air temperature (`air.sig995`), available every 6 hours on a 2.5° grid.
- **CPC Global Unified Gauge-based Analysis** of daily precipitation, available daily on a 0.5° grid.

Both datasets cover the United States, differ in temporal cadence and spatial resolution, and already share the same 0–360° longitude convention. Our goal is to create a merged dataset containing daily mean temperature and precipitation on a common 0.5° grid.


## 2. Open the remote datasets

`xarray` can lazily stream data from remote THREDDS servers using `open_dataset`. We apply simple chunking to keep operations memory-friendly.


In [ ]:
air = xr.open_dataset(TEMP_URL, chunks={"time": 8})
precip = xr.open_dataset(PRECIP_URL, chunks={"time": 31})

air, precip


## 3. Spatial and temporal subsetting

To reduce data volume, we focus on June 2020 over the continental United States.

Because both products express longitude from 0° to 360°, the continental US spans roughly 230°E–300°E. The helper below adapts to each dataset's coordinate ordering.


In [ ]:
def oriented_slice(coord, bounds):
    values = coord.to_numpy()
    start, end = bounds
    forward = float(values[0]) < float(values[-1])
    return slice(start, end) if forward else slice(end, start)

def select_lon_bounds(coord):
    values = coord.to_numpy()
    bounds = LON_RANGE_360 if float(values.min()) >= 0 else LON_RANGE_180
    return oriented_slice(coord, bounds)

air_us = air.sel(
    time=TIME_RANGE,
    lat=oriented_slice(air.lat, LAT_RANGE),
    lon=select_lon_bounds(air.lon),
)
precip_us = precip.sel(
    time=TIME_RANGE,
    lat=oriented_slice(precip.lat, LAT_RANGE),
    lon=select_lon_bounds(precip.lon),
)

air_us, precip_us


## 4. Match temporal resolution with `xr.resample`

The air temperature is reported every six hours. We compute daily means to match the precipitation cadence using `xr.resample(time="1D")`.


In [ ]:
air_daily = air_us["air"].resample(time="1D").mean()
air_daily


## 5. Match spatial resolution with `xr.interp`

The precipitation grid is denser (0.5°). Interpolate the coarser air temperature field onto the precipitation grid.


In [ ]:
air_interp = air_daily.interp(lat=precip_us.lat, lon=precip_us.lon)
air_interp


## 6. Merge the harmonized datasets

Once the data share the same grid and cadence, `xr.merge` combines them into a single dataset.


In [ ]:
air_ds = air_interp.to_dataset(name="air_temperature")
precip_ds = precip_us["precip"].to_dataset(name="daily_precip")

merged = xr.merge([air_ds, precip_ds])
merged


## 7. Quick sanity checks

We confirm that both variables now have identical dimensions and a few simple statistics look reasonable.


In [ ]:
print(dict(merged.dims))
temp_min = float(merged.air_temperature.min().compute())
temp_max = float(merged.air_temperature.max().compute())
precip_min = float(merged.daily_precip.min().compute())
precip_max = float(merged.daily_precip.max().compute())
print(temp_min, temp_max)
print(precip_min, precip_max)


## 8. Example visualization (optional)

Plotting provides a visual confirmation that the merged grid is consistent.
The cartopy map below renders both fields on a shared projection, and the scatter
plot compares collocated temperature and precipitation values.


In [ ]:
if ccrs is None:
    raise ImportError("Install cartopy to render the projected map (e.g., pip install cartopy).")

temp_slice = merged.air_temperature.isel(time=0).compute()
precip_slice = merged.daily_precip.isel(time=0).compute()

west, east = (LON_RANGE_360[0] - 360, LON_RANGE_360[1] - 360)
south, north = LAT_RANGE

fig, axes = plt.subplots(
    1,
    2,
    figsize=(12, 4),
    subplot_kw={"projection": ccrs.PlateCarree()},
    constrained_layout=True,
)

_ = temp_slice.plot(
    ax=axes[0],
    transform=ccrs.PlateCarree(),
    cmap="coolwarm",
    robust=True,
    add_colorbar=True,
    cbar_kwargs={"shrink": 0.75, "label": "K"},
)
axes[0].set_title("Surface air temperature (first day)")
axes[0].set_extent([west, east, south, north], crs=ccrs.PlateCarree())
axes[0].coastlines(resolution="50m", linewidth=0.8)
if cfeature is not None:
    try:
        axes[0].add_feature(cfeature.STATES.with_scale("50m"), linewidth=0.4, edgecolor="black")
    except Exception:
        pass

_ = precip_slice.plot(
    ax=axes[1],
    transform=ccrs.PlateCarree(),
    cmap="Blues",
    robust=True,
    add_colorbar=True,
    cbar_kwargs={"shrink": 0.75, "label": "mm/day"},
)
axes[1].set_title("Daily precipitation (first day)")
axes[1].set_extent([west, east, south, north], crs=ccrs.PlateCarree())
axes[1].coastlines(resolution="50m", linewidth=0.8)
if cfeature is not None:
    try:
        axes[1].add_feature(cfeature.STATES.with_scale("50m"), linewidth=0.4, edgecolor="black")
    except Exception:
        pass

stacked = merged[["air_temperature", "daily_precip"]].stack(point=("time", "lat", "lon"))
stacked = stacked.dropna(dim="point", how="any").compute()
temp_vals = stacked.air_temperature.values
precip_vals = stacked.daily_precip.values

scatter_fig, scatter_ax = plt.subplots(figsize=(5, 4), constrained_layout=True)
scatter_ax.scatter(temp_vals, precip_vals, s=8, alpha=0.25, edgecolor="none")
scatter_ax.set_xlabel("Surface air temperature (K)")
scatter_ax.set_ylabel("Daily precipitation (mm/day)")
scatter_ax.set_title("Collocated temperature vs precipitation")


## 9. Save merged dataset (optional)

Persist the merged result locally if you plan further analysis.


In [ ]:
# merged.to_netcdf("data/merged_air_precip_june2020.nc")
